# Lowlevel Simulation

In [ ]:
from Go2Py.sim.mujoco import Go2Sim
import numpy as np

In [ ]:
robot = Go2Sim(mode='lowlevel')
robot.standUpReset()

In [ ]:
robot.getJointStates()

In [ ]:
import mujoco
import time
robot.standUpReset()
for i in range(100000):
    state = robot.getJointStates()
    tau = 20*np.eye(12)@(robot.q0 - state['q']).reshape(12,1)
    robot.setCommands(np.zeros(12), np.zeros(12), np.zeros(12), np.zeros(12), tau)
    robot.step()

# Highlevel Simulation

In [1]:
from Go2Py.sim.mujoco import Go2Sim
import numpy as np

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
robot = Go2Sim(mode='highlevel')
robot.standUpReset()

p_gains: [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


In [3]:
import time
robot.standUpReset
for i in range(10000):
    robot.step(0,0,0., step_height=0,kp=[2, 0.5, 0.5], ki=[0.02, 0.01, 0.01])
    time.sleep(robot.dt)

frq: 0.06607245464787131 Hz
frq: 33.44393324455997 Hz
frq: 51.28137914170436 Hz
frq: 57.72269242943451 Hz
frq: 55.13307744886692 Hz
frq: 54.144503969534625 Hz
frq: 54.06006238238857 Hz
frq: 54.02315846417393 Hz
frq: 52.84362243612357 Hz
frq: 56.53615139914811 Hz
frq: 57.84608594913664 Hz
frq: 54.998610054811046 Hz
frq: 52.402598700649676 Hz
frq: 55.65542315755951 Hz
frq: 53.71872078279691 Hz
frq: 57.66079652465597 Hz
frq: 53.88919724534896 Hz
frq: 56.06533798505567 Hz
frq: 57.76641692375496 Hz
frq: 55.159904785702075 Hz


In [4]:
robot.getLidarData()

{'pcd': array([[-1.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [-9.99981138e-01, -6.14188251e-03, -0.00000000e+00],
        [-9.99924555e-01, -1.22835333e-02, -0.00000000e+00],
        ...,
        [-9.99924555e-01,  1.22835333e-02, -0.00000000e+00],
        [-9.99981138e-01,  6.14188251e-03, -0.00000000e+00],
        [-1.00000000e+00,  2.44929360e-16, -0.00000000e+00]]),
 'geomid': array([-1, -1, -1, ..., -1, -1, -1], dtype=int32),
 'dist': array([-1., -1., -1., ..., -1., -1., -1.])}